# imports

In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
from scipy import spatial
import glob
import os

# all zipcodes

In [2]:
zipcode_df = pd.read_csv('data/processed/zipcode.csv',index_col=[0])

zipcode_df = zipcode_df.rename(columns={
    'Latitude':'latitude',
    'Longitude':'longitude',
})

# all df_pararius

In [3]:
list_of_files = glob.glob("data/temp/df_pararius_*.csv") # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
df_pararius = pd.read_csv(latest_file, index_col=[0])
df_pararius.sample()

,price,address,street,agency,irl,image,surface-area,status,number-of-rooms,interior,garden-surface-area,construction-period,plot-size,postcode,date
4860,2100.0,2561 DN Den Haag (Valkenboskwartier),Apartment Fahrenheitstraat 11,Tweelwonen.nl Rotterdam,/apartment-for-rent/den-haag/ffc29911/fahrenhe...,https://casco-media-prod.global.ssl.fastly.net...,102,NaN,6,Upholstered,NaN,NaN,NaN,2561DN,2021-10-12


# What is new?

In [4]:
temp1 = df_pararius['postcode'].unique()
temp2 = zipcode_df['postcode'].unique()
cep = list(set(temp1) - set(temp2))
print(len(cep))

30


In [5]:
geolocator = Nominatim(user_agent="myGeocoder")
post_code=[]
erro_code=[]

for zipcode in cep[:]:
    try:
        dict_zip = {
            'zipcode':zipcode, 
            'geolocator':geolocator.geocode(zipcode).address, 
            'latitude':geolocator.geocode(zipcode).latitude, 
            'longitude':geolocator.geocode(zipcode).longitude, 
        }
        post_code.append(dict_zip)

    except:
        erro_code.append(zipcode)

In [6]:
zipcode = pd.DataFrame(post_code)
splited = zipcode['geolocator'].str.split(",",expand=True)
zipcode['Plaats'] = splited[0]
zipcode['Provincie'] = splited[1]
zipcode = zipcode.drop(['geolocator'],1)

<ipython-input-6-b9944e94601d>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  zipcode = zipcode.drop(['geolocator'],1)


# Manual Coordinates

## Not Found

In [7]:
erro_code

['2595BW', '2595BX']

# Save list zipcodes

In [8]:
zipcode_df = pd.concat([zipcode_df,zipcode],0).drop_duplicates(subset=['postcode']).reset_index(drop=True)

<ipython-input-8-815d2116adb3>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  zipcode_df = pd.concat([zipcode_df,zipcode],0).drop_duplicates(subset=['postcode']).reset_index(drop=True)


In [9]:
zipcode_df = zipcode_df.drop(['zipcode'], axis=1)

# Add Coord into df_pararius

In [10]:
zipcode_df = zipcode_df.rename(columns={'zipcode':'postcode'})
df_pararius_coo=pd.merge(df_pararius, zipcode_df, on='postcode',how='left')

# organizing before saving

In [11]:
df_pararius_coo = df_pararius_coo.fillna(0).drop_duplicates(subset=['irl']).reset_index(drop=True)

In [12]:
df_pararius_coo.loc[df_pararius_coo['interior']==0, 'interior'] = 'Upholstered'
df_pararius_coo.loc[df_pararius_coo['status']==0, 'status'] = 'Free'

In [13]:
df_pararius_coo['city'] = df_pararius_coo['irl'].str.split("/",expand=True)[2].str.replace("-", " ").str.title()

In [14]:
# prepare link on streamlit
df_pararius_coo['url'] = 'https://www.pararius.com' + df_pararius_coo['irl']
df_pararius_coo['link'] = "<a target='_blank' href=" + (df_pararius_coo['url']).astype(str) + ">" + (df_pararius_coo['street']).astype(str) + "</a>"
df_pararius_coo['img'] = "<a href=" + df_pararius_coo['url'] + " target='blank'><img src=" + df_pararius_coo['image'] + "title='rent' width='150' height='100'/></a>"

# save data

In [15]:
df_pararius_coo.to_csv('app/df_coo_pararius.csv')
zipcode_df.to_csv('data/processed/zipcode.csv')